### importing libraries and modules

In [ ]:
import torch
import numpy as np
import scipy
import matplotlib.pyplot as plt
from forward_config import device, Pk, t_end, L
from model import DNN


### train model

In [ ]:
import matplotlib.cm as cm
import time
from torch.optim.lr_scheduler import ExponentialLR


device = torch.device(device)

lr = 0.001
epochs = 35000

u = torch.linspace(0, 1, 50)
t = torch.linspace(0, 1, 50)
###
grid = torch.cartesian_prod(u,t).float().to(device)

# p(u,0)=1
bnd1 = torch.cartesian_prod(u, torch.tensor([0.])).float().to(device)
bndval1 = torch.tensor([1.]).to(device)

#bnd2(u(-1), t)=1
bnd2 = torch.cartesian_prod(torch.tensor([1.]), t).float().to(device)
bndval2 = torch.tensor([1.]).to(device)

#bnd3
bnd3 = torch.cartesian_prod(torch.tensor([0.]), t).float().to(device)

NN = torch.nn.Sequential(
            torch.nn.Linear(2, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 50),
            torch.nn.Tanh(),
            torch.nn.Linear(50, 1))


# Initialize neural network
model = DNN(NN).to(device)


# Loss and optimizer
optimizer = torch.optim.Adam(model.net.parameters(), lr)
scheduler = ExponentialLR(optimizer, gamma=0.9)

    # Train PINNs
for epoch in range(epochs):
    model.train()

    def closure():
        optimizer.zero_grad()
        loss_pde = model.loss_pde(grid)
        loss_bnd1 = model.loss_dirichlet(bnd1, bndval1).reshape(-1)
        loss_bnd2 = model.loss_dirichlet(bnd2, bndval2).reshape(-1)
        loss_bnd3 = model.loss_operator(bnd3).reshape(-1)

        loss = 10*loss_pde + 1000*loss_bnd1.mean() + 100*(torch.hstack([loss_bnd2, loss_bnd3])).mean()

        loss.backward()
        return loss


    # Optimize loss function
    loss_pde = optimizer.step(closure)

    loss_value = loss_pde.item() if not isinstance(loss_pde, float) else loss_pde


    if epoch%3000 == 0:
        scheduler.step()

    if epoch%1500==0:
        print(f'epoch {epoch}: loss {loss_value:.6f}')


### compare with finite diffs

In [ ]:
from reference import finite_diffs

P, x, t = finite_diffs(50, 50)

u = torch.linspace(0, 1, 50)
t = torch.linspace(0, 1, 50)

grid1 = torch.cartesian_prod(u, torch.tensor([t[1].item()])).float().to(device)
grid2 = torch.cartesian_prod(u, torch.tensor([t[10].item()])).float().to(device)
grid3 = torch.cartesian_prod(u, torch.tensor([t[20].item()])).float().to(device)
grid4 = torch.cartesian_prod(u, torch.tensor([t[30].item()])).float().to(device)
grid5 = torch.cartesian_prod(u, torch.tensor([t[40].item()])).float().to(device)
grid6 = torch.cartesian_prod(u, torch.tensor([t[49].item()])).float().to(device)

import matplotlib
plt.figure()
plt.figure(figsize=(8, 6))
font = {'serif' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 13}

matplotlib.rc('font', **font)

plt.plot(u, model(grid1).cpu().detach().numpy(),'o', c='k', mfc='w', mec='k',ms=6, label = 'PINN model')
plt.plot(u, P[:,1]/Pk, '--', c='k', lw=2, label = 'Finite difference model')
plt.plot(u, model(grid2).cpu().detach().numpy(),'o', c='k', mfc='w', mec='k',ms=6)
plt.plot(u, P[:,10]/Pk, '--', c='k', lw=2)
plt.plot(u, model(grid3).cpu().detach().numpy(), 'o', c='k', mfc='w', mec='k',ms=6)
plt.plot(u, P[:,20]/Pk, '--', c='k', lw=2)
plt.plot(u, model(grid4).cpu().detach().numpy(), 'o', c='k', mfc='w', mec='k',ms=6)
plt.plot(u, P[:,30]/Pk, '--', c='k', lw=2)
plt.plot(u, model(grid5).cpu().detach().numpy(), 'o', c='k', mfc='w', mec='k',ms=6)
plt.plot(u, P[:,40]/Pk, '--', c='k', lw=2)
plt.plot(u, model(grid6).cpu().detach().numpy(), 'o', c='k', mfc='w', mec='k',ms=6)
plt.plot(u, P[:,49]/Pk, '--', c='k', lw=2)
plt.xlabel('x/L, m/m')
plt.ylabel('$P/P_0$, MPa/MPa')
plt.legend(loc='lower right')
plt.grid(True)
#plt.savefig('PINN_FD.pdf',dpi = 1000)

### relative rmse confidence bound

In [ ]:
def fd_solution(grid):
    """Interpolate finite difference solution to another grid"""
    P, x, t = finite_diffs(300, 200)

    P = (P / Pk).reshape(-1)

    x = x / L

    t = t / t_end

    grid_fd = torch.cartesian_prod(torch.from_numpy(x), torch.from_numpy(t))

    u = scipy.interpolate.griddata(grid_fd, P, grid, method='nearest')

    return u

rel_rmse = []

for i in range(10):

    device = torch.device(device)

    lr = 0.001
    epochs = 35000

    u = torch.linspace(0, 1, 50)
    t = torch.linspace(0, 1, 50)
    ###
    grid = torch.cartesian_prod(u,t).float().to(device)

    # p(u,0)=1
    bnd1 = torch.cartesian_prod(u, torch.tensor([0.])).float().to(device)
    bndval1 = torch.tensor([1.]).to(device)

    #bnd2(u(-1), t)=1
    bnd2 = torch.cartesian_prod(torch.tensor([1.]), t).float().to(device)
    bndval2 = torch.tensor([1.]).to(device)

    #bnd3
    bnd3 = torch.cartesian_prod(torch.tensor([0.]), t).float().to(device)

    NN = torch.nn.Sequential(
                torch.nn.Linear(2, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 50),
                torch.nn.Tanh(),
                torch.nn.Linear(50, 1))


    # Initialize neural network
    model = DNN(NN).to(device)


    # Loss and optimizer
    optimizer = torch.optim.Adam(model.net.parameters(), lr)
    scheduler = ExponentialLR(optimizer, gamma=0.9)

        # Train PINNs
    for epoch in range(epochs):
        model.train()

        def closure():
            optimizer.zero_grad()
            loss_pde = model.loss_pde(grid)
            loss_bnd1 = model.loss_dirichlet(bnd1, bndval1).reshape(-1)
            loss_bnd2 = model.loss_dirichlet(bnd2, bndval2).reshape(-1)
            loss_bnd3 = model.loss_operator(bnd3).reshape(-1)

            loss = 10*loss_pde + 1000*loss_bnd1.mean() + 100*(torch.hstack([loss_bnd2, loss_bnd3])).mean()

            loss.backward()
            return loss


        # Optimize loss function
        loss_pde = optimizer.step(closure)


        if epoch%3000 == 0:
            scheduler.step()

    pinn_sol = model(grid).detach().cpu().numpy().reshape(-1)

    fd_sol, xx, tt = finite_diffs(50, 50)

    fd_sol = (fd_sol / Pk).reshape(-1)

    rmse = np.sqrt(np.sum((fd_sol - pinn_sol)**2)) / np.sqrt(np.sum(fd_sol**2))
    print("relative RMSE= ", rmse)
    rel_rmse.append(rmse)

In [ ]:
import scipy.stats as st 
 
# create 95% confidence interval 
conf_inerval = st.t.interval(confidence=0.95, df=len(rel_rmse)-1, 
                            loc=np.mean(rel_rmse), 
                            scale=st.sem(rel_rmse))

conf_inerval